In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval


In [2]:
ratings = pd.read_csv("../cosineSimilarity/ratings_small.csv")
movieProperties = ratings.groupby('movieId').agg({'rating': [np.size, np.mean]})
movieProperties.head()

rating          
          size      mean
movieId                 
1          247  3.872470
2          107  3.401869
3           59  3.161017
4           13  2.384615
5           56  3.267857

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movieId,
1,0.723529
2,0.311765
3,0.170588
4,0.035294
5,0.161765


In [4]:
movies = pd.read_csv("../cosineSimilarity/movies_metadata.csv")
movies['year'] = pd.to_datetime(movies['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
movies = movies.drop([19730, 29503, 35587])
movies = movies.rename(columns={'id':'tmdbId'})
links = pd.read_csv("../cosineSimilarity/small_links.csv")
movies["tmdbId"] = movies["tmdbId"].astype("int")


fulldata = pd.merge(movies,links)
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
genres = movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
genres = genres.unique()


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/mr/3szmdm1d3sj80jkk3tl1c_t40000gp/T/ipykernel_4704/1345081028.py:11: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  genres = movies.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


In [5]:
movieNormalizedNumRatings = movieNormalizedNumRatings.reset_index()
# Function to convert  
def checkGenre(s): 
    
    lst = []
    
    # traverse in the string  
    for ele in genres: 
        if (ele in s):
            lst.append(1)
        else:
            lst.append(0)

    
    # return string  
    return lst


fulldata["binaryGenre"] = fulldata["genres"].apply(lambda x: checkGenre(x))


In [6]:
fulldata = pd.merge(fulldata,movieNormalizedNumRatings)

In [7]:
fulldata["props"] = fulldata["movieId"].apply(lambda x: movieProperties.loc[x].rating.get("mean"))

In [10]:
prepocessed_data = fulldata[["title","binaryGenre","size","props"]]

In [18]:
for record in prepocessed_data.iteritems():
    print(record)

('title', 0                                               Toy Story
1                                                 Jumanji
2                                        Grumpier Old Men
3                                       Waiting to Exhale
4                             Father of the Bride Part II
                              ...                        
9037                         Sharknado 4: The 4th Awakens
9038                       The Last Brickmaker in America
9039                                               Rustom
9040                                         Mohenjo Daro
9041    The Beatles: Eight Days a Week - The Touring Y...
Name: title, Length: 9042, dtype: object)
('binaryGenre', 0       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1       [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
4       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
    